In [1]:
ENV["JULIA_REVISE"] = "manual" 
using Revise
using Probabilistic
using BenchmarkTools

In [10]:
revise()

In [12]:
brownian = @model N begin
    σ ~ Beta(2, 3)
    x = zeros(N)
    for i = 2:N
       x[i] ~ Normal(x[i-1], σ^2)
    end
end

N = 20
X = cumsum((0.75^2)*randn(N))

@benchmark sample(brownian(N) |> condition(:x => X) |> observe(:σ), Metropolis(1_000))

BenchmarkTools.Trial: 
  memory estimate:  5.80 MiB
  allocs estimate:  211434
  --------------
  minimum time:     144.028 ms (0.00% GC)
  median time:      145.503 ms (0.00% GC)
  mean time:        147.298 ms (1.12% GC)
  maximum time:     154.857 ms (4.00% GC)
  --------------
  samples:          34
  evals/sample:     1

In [9]:
mix = @model N begin
    prob ~ Beta(2, 2)

    mean0 ~ Normal(-1,1)
    mean1 ~ Normal(1,1)

    x = zeros(N)
    for i = 1:N
        x[i] ~ BiMix(prob, Normal(mean0, 0.2), Normal(mean1, 0.2))
    end
end

obs = [-1.7, -1.8, -2.01, -2.4, 1.9, 1.8]
N = length(obs)

@benchmark sample(mix(N) |> condition(:x => obs) |> observe(:mean0, :mean1), Metropolis(10_000))

BenchmarkTools.Trial: 
  memory estimate:  28.71 MiB
  allocs estimate:  958073
  --------------
  minimum time:     433.412 ms (1.34% GC)
  median time:      442.215 ms (1.39% GC)
  mean time:        442.403 ms (1.73% GC)
  maximum time:     451.291 ms (2.62% GC)
  --------------
  samples:          12
  evals/sample:     1